In [ ]:
# Install required packages (run once)
# !pip install gTTS ipython

from gtts import gTTS
from IPython.display import Audio, display
import os

# ======================
# CONFIGURATION
# ======================
DISABLE_AUDIO = False  # Set to True to disable audio (faster execution)
CACHE_AUDIO = True    # Set to False to always regenerate audio
USD_TO_ETB = 138.80   # Current exchange rate (1 USD = 138.80 ETB)

# ======================
# CURRENCY FORMATTING
# ======================
def format_currency(amount_usd, lang):
    """Convert USD to local currency display"""
    if lang in ['am', 'om', 'ti']:  # Ethiopian languages
        amount_local = amount_usd * USD_TO_ETB
        return f"{amount_local:,.2f} ብር"  # Ethiopian Birr
    else:  # English/other
        return f"${amount_usd:,.2f}"

# ======================
# CREDIT TERM EXPLANATIONS
# ======================
credit_terms = {
    "interest rate": {
        "am": "የወለድ መጠን ማንኛውም የተበደለ ገንዘብ የሚከፈልበት ዋጋ ነው፣ በመደበኛ ሁኔታ እንደ መቶኛ በመጠቀም ይታያል።",
        "om": "Gatii dabalataa maallaqaa liqeeffameef kaffalamuudha, baayʼinaan dhibbeentaan ibsama.",
        "ti": "ዝርከብ ዋጋ እዩ ክፍሊት ኣብ ኣእታዊ ዘርእስቲ ምኽንያት እዩ።",
        "en": "Interest rate is the cost of borrowing money, usually expressed as a percentage."
    },
    "loan": {
        "am": "ብድር ማለት እርስዎ የሚበደሉትን ገንዘብ ከወለድ ጋር መመለስ ያስፈልጋል ማለት ነው።",
        "om": "Liqiin maallaqa fudhatamuudha kan dabalataan deebisamuu qabu.",
        "ti": "ብድሪ እዩ ክትዕቀብ ከም ንምኽንያት ክትክፈል ኣለኻ።",
        "en": "A loan is money you borrow that you have to pay back, usually with interest."
    },
    "repayment": {
        "am": "ክፍያ ማለት በጊዜ ላይ በትንሽ ትንሽ መጠን የተበደለውን ገንዘብ መመለስ ማለት ነው።",
        "om": "Deebisuun jechuun maallaqa liqeeffame xiqqaan xiqqaan yeroo dheeraaf deebisuudha.",
        "ti": "መክፈሊ ምልእኽቲ እዩ እቲ ብትንሽ ብትንሽ ገንዘብ ንትሕትና ክትምለስ እዩ።",
        "en": "Repayment means paying back the borrowed money in small amounts over time."
    },
    "credit score": {
        "am": "የብድር ነጥብ ማለት ብድሩን ለመከፈል የምትችሉበትን እድል የሚገልጽ ቁጥር ነው።",
        "om": "Lakkoofsi krediitii liqaa deebifamuu dandaʼu agarsiisa.",
        "ti": "ስኮር ክሬዲት ቁጽሪ እዩ ክንከፍል ከም እተወሃበ ኣርእስቲ ዝወከስ እዩ።",
        "en": "Credit score is a number that indicates your ability to repay a loan."
    }
}

# ======================
# TRANSLATIONS (UPDATED WITH CURRENCY)
# ======================
translations = {
    "am": {
        "eligible": "ሰላም {name}፣ እስከ {amount} ድረስ ብድር ለመቀበል የሚችሉ ነዎት።",
        "not_eligible": "ሰላም {name}፣ ይቅርታ፣ በዚህ ጊዜ ብድር ለመቀበል አትችሉም።",
        "no_term": "ይቅርታ፣ ስለዚህ ቃል መረጃ የለኝም።",
        "goodbye": "👋 ደህና ሁኑ! MicroLoanMate እናመሰግናለን።",
        "learn_terms": "በየትም የብድር ቃል ላይ መረጃ ማግኘት ይፈልጋሉ? (አዎ/አይደለም): ",
        "term_prompt": "🔎 የብድር ቃል ያስገቡ (ለምሳሌ፡ �ለድ መጠን፣ ብድር፣ ክፍያ፣ �እለድ ነጥብ): ",
        "try_again": "🔄 እንደገና ለመሞከር ይፈልጋሉ? (አዎ/አይደለም): ",
        "income_prompt": "💵 ወርሃዊ ገቢዎ በብር ስንት ነው? (1 ዶላር = 138.80 ብር): ",
        "currency_note": "ማስታወሻ፡ 1 �ላር = 138.80 ብር"
    },
    "om": {
        "eligible": "Akkam {name}, hanga {amount} liqaa argachuuf carraa qabda.",
        "not_eligible": "Akkam {name}, dhiifama ammaaf liqaa argachuuf hin dandeessu.",
        "no_term": "Dhiifama, waa'ee jecha sanaa odeeffannoo hin qabu.",
        "goodbye": "👋 Nagaan koo! MicroLoanMate fayyadamteetta.",
        "learn_terms": "Termii liqii irratti baruu barbaadda? (eeyyee/lakki): ",
        "term_prompt": "🔎 Termii liqii galchi (fakkeenya: gatii dabalataa, liqii, deebisuu, lakkoofsa krediitii): ",
        "try_again": "🔄 Irra deebi'uu barbaadda? (eeyyee/lakki): ",
        "income_prompt": "💵 Galii ji'aa Birrii meeqa? (1 Dollar = 138.80 Birrii): ",
        "currency_note": "Yaadadhu: 1 Dollar = 138.80 Birrii"
    },
    "ti": {
        "eligible": "ሰላም {name}፣ ክሳዕ {amount} ድሕሪ ምዝራብ ብድሪ ክትወስድ ትኽእል ኢኻ።",
        "not_eligible": "ሰላም {name}፣ ይቕሬታ፣ ኣብዚ እዋን ንብድሪ ክትወስድ ኣይትኽእልን ኢኻ።",
        "no_term": "ይቕሬታ፣ ብዛዕባ እቲ ቃል ሓበሬታ የብለይን።",
        "goodbye": "👋 ደሓን እትምለስ! MicroLoanMate ኣመስግናኻ።",
        "learn_terms": "ብዛዕባ ቃላት ክሬዲት ክትፈልጥ ትደሊዶ? (እወ/ኣይፋል): ",
        "term_prompt": "🔎 ቃል ክሬዲት ኣእትው (ንኣብነት: ዋጋ ወለድ፣ ብድሪ፣ ክፍሊት፣ ነጥብ ክሬዲት): ",
        "try_again": "🔄 ደጊምካ ክትፈትን ትደሊዶ? (እወ/ኣይፋል): ",
        "income_prompt": "💵 ወርሓዊ ገቢካ ብብር ክንደይ እዩ? (1 ዶላር = 138.80 ብር): ",
        "currency_note": "ማሕበር: 1 ዶላር = 138.80 ብር"
    },
    "en": {
        "eligible": "Hello {name}, you're eligible for a loan up to {amount}.",
        "not_eligible": "Hello {name}, unfortunately you're not eligible for a loan at this time.",
        "no_term": "Sorry, I don't have information about that term.",
        "goodbye": "👋 Goodbye! Thanks for using MicroLoanMate.",
        "learn_terms": "Do you want to learn about any credit terms? (yes/no): ",
        "term_prompt": "🔎 Enter a credit term (e.g., interest rate, loan, repayment, credit score): ",
        "try_again": "🔄 Do you want to try again? (yes/no): ",
        "income_prompt": "💵 What is your monthly income in Birr? (1 USD = 138.80 ETB): ",
        "currency_note": "Note: 1 USD = 138.80 ETB"
    }
}

# ======================
# QUESTIONS (UPDATED FOR LOCAL CURRENCY)
# ======================
questions = {
    "name": {
        "am": "👤 ስምዎ ማን ነው?",
        "om": "👤 Maqaan kee eenyu?",
        "ti": "👤 ስምካ እንታይ ደ?",
        "en": "👤 What is your name?"
    },
    "income": {
        "am": translations["am"]["income_prompt"],
        "om": translations["om"]["income_prompt"],
        "ti": translations["ti"]["income_prompt"],
        "en": translations["en"]["income_prompt"]
    },
    "dependents": {
        "am": "👨‍👩‍👧‍👦 ምን ብዛት ሰዎችን ይደግፋሉ?",
        "om": "👨‍👩‍👧‍👦 Nama meeqa deeggartuu?",
        "ti": "👨‍👩‍👧‍👦 ክንደይ ሰባት ኣብ ሓጎሰካ ኣሎም?",
        "en": "👨‍👩‍👧‍👦 How many people do you support?"
    },
    "past_repayment": {
        "am": "📊 ያለፈው የብድር ክፍያዎ ታሪክ እንዴት ነበር?",
        "om": "📊 Seenaa deebii liqaa kee akkam ture?",
        "ti": "📊 ታሪኽ ናይ ብድሪ መክፈሊ እንታይ ነበረ?",
        "en": "📊 How was your past repayment history?"
    }
}

# ======================
# REPAYMENT OPTIONS
# ======================
repayment_options = {
    "am": {"good": "ጥሩ", "average": "መካከለኛ", "poor": "መከላከያ"},
    "om": {"good": "gaarii", "average": "giddugaleessa", "poor": "hamaa"},
    "ti": {"good": "ጽቡቕ", "average": "መካከለኛ", "poor": "ክፉ"},
    "en": {"good": "good", "average": "average", "poor": "poor"}
}

# ======================
# CORE FUNCTIONS (UPDATED FOR CURRENCY)
# ======================
def check_eligibility(income_etb, dependents, past_repayment):
    """Determine loan eligibility based on financial factors"""
    income_usd = income_etb / USD_TO_ETB
    
    if income_usd >= 100 and dependents <= 3 and past_repayment == 'good':
        return True, 15000  # $15,000
    elif income_usd >= 50 and past_repayment == 'average':
        return True, 7500   # $7,500
    else:
        return False, 0

def speak(text, lang='en'):
    """Optimized TTS function with caching"""
    if DISABLE_AUDIO:
        print(f"[{lang.upper()}]: {text}")
        return
    
    lang_map = {'en': 'en', 'am': 'am', 'om': 'en', 'ti': 'en'}
    tts_lang = lang_map.get(lang, 'en')
    
    if CACHE_AUDIO:
        filename = f"audio_{hash(text + lang)}.mp3"
        if os.path.exists(filename):
            display(Audio(filename, autoplay=True))
            return
    
    try:
        tts = gTTS(text=text, lang=tts_lang, slow=False)
        if not CACHE_AUDIO:
            filename = "temp_audio.mp3"
        tts.save(filename)
        display(Audio(filename, autoplay=True))
        if not CACHE_AUDIO:
            os.remove(filename)
    except Exception as e:
        print(f"[{lang.upper()} AUDIO - Fallback]: {text}")

# ======================
# MAIN APPLICATION FLOW (UPDATED)
# ======================
def microloan_advisor(lang):
    print("\n📱 Welcome to MicroLoanMate - Your Private Loan Advisor\n")
    print("="*50)
    print(translations[lang]["currency_note"])
    
    # Collect user information
    name = input(questions["name"].get(lang, questions["name"]["en"]) + " ")
    income_etb = float(input(questions["income"].get(lang, questions["income"]["en"]) + " "))
    dependents = int(input(questions["dependents"].get(lang, questions["dependents"]["en"]) + " "))

    # Handle repayment history
    options = repayment_options[lang]
    prompt = f"📊 {questions['past_repayment'][lang]} ({options['good']}/{options['average']}/{options['poor']}): "
    past_repayment_input = input(prompt).strip().lower()
    
    # Map input to standard values
    reverse_map = {v.lower(): k for k, v in options.items()}
    past_repayment = reverse_map.get(past_repayment_input, "poor")

    # Check eligibility
    eligible, usd_amount = check_eligibility(income_etb, dependents, past_repayment)
    formatted_amount = format_currency(usd_amount, lang)

    # Display results
    print("\n" + "="*50)
    print("✅ Eligibility Result:")
    if eligible:
        message = translations[lang]['eligible'].format(name=name, amount=formatted_amount)
    else:
        message = translations[lang]['not_eligible'].format(name=name)
    
    print(message)
    speak(message, lang)

    # Offer credit term explanations
    explain = input("\n❓ " + translations[lang]['learn_terms']).strip().lower()
    if explain in ['yes', 'አዎ', 'eeyyee', 'እወ']:
        term = input(translations[lang]['term_prompt']).strip().lower()
        explanation = credit_terms.get(term, {}).get(lang, translations[lang]['no_term'])
        print("\n📘 Explanation:")
        print(explanation)
        speak(explanation, lang)

# ======================
# MAIN LOOP
# ======================
def main_loop():
    while True:
        print("\n" + "="*50)
        lang = input("🌐 Choose your language (en=English, am=Amharic, om=Oromo, ti=Tigrigna): ").strip().lower()
        
        if lang not in translations:
            print("⚠️ Language not supported. Defaulting to English.")
            lang = 'en'

        microloan_advisor(lang)

        again = input("\n" + translations[lang]['try_again']).strip().lower()
        if again not in ['yes', 'አዎ', 'eeyyee', 'እወ']:
            print("\n" + "="*50)
            print(translations[lang]['goodbye'])
            print("="*50 + "\n")
            break

# ======================
# RUN THE PROGRAM
# ======================
if __name__ == "__main__":
    main_loop()

🌐 Choose your language (en=English, am=Amharic, om=Oromo, ti=Tigrigna):  om



📱 Welcome to MicroLoanMate - Your Private Loan Advisor

Yaadadhu: 1 Dollar = 138.80 Birrii


👤 Maqaan kee eenyu?  Jabessaa
💵 Galii ji'aa Birrii meeqa? (1 Dollar = 138.80 Birrii):   2000000
👨‍👩‍👧‍👦 Nama meeqa deeggartuu?  3
📊 📊 Seenaa deebii liqaa kee akkam ture? (gaarii/giddugaleessa/hamaa):  gaarii



✅ Eligibility Result:
Akkam Jabessaa, hanga 2,082,000.00 ብር liqaa argachuuf carraa qabda.



❓ Termii liqii irratti baruu barbaadda? (eeyyee/lakki):  eeyyee
🔎 Termii liqii galchi (fakkeenya: gatii dabalataa, liqii, deebisuu, lakkoofsa krediitii):  deebisuu



📘 Explanation:
Dhiifama, waa'ee jecha sanaa odeeffannoo hin qabu.
